In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, FreqDist

from wordcloud import WordCloud

pd.options.display.float_format = '{:.3f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 300 # max width of a column

%matplotlib inline

In [18]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/gburgess/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [2]:
jigsaw_df = pd.read_csv('../data/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv')

In [3]:
is_toxic = (jigsaw_df['toxicity'] >= 0.5) | (jigsaw_df['severe_toxicity'] >= 0.1) | \
            (jigsaw_df['obscene'] >= 0.25) | (jigsaw_df['sexual_explicit'] >= 0.25) | \
            (jigsaw_df['identity_attack'] >= 0.25) | (jigsaw_df['insult'] >= 0.25) | \
            (jigsaw_df['threat'] >= 0.10)

In [4]:
new_target_df = pd.DataFrame()
new_target_df[['id', 'comment_text']] = jigsaw_df[['id', 'comment_text']].copy()
new_target_df['target'] = (is_toxic).astype(int)
new_target_df.info(show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999516 entries, 0 to 1999515
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   id            1999516 non-null  int64 
 1   comment_text  1999515 non-null  object
 2   target        1999516 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 45.8+ MB


In [5]:
# remove single row with NaN in `comment_text` (probably there just for fun!)
new_target_df.dropna(inplace=True)

In [6]:
del jigsaw_df

In [7]:
is_target = (new_target_df['target'] == 1)
num_targets = sum(is_target)
num_targets

338409

In [8]:
majority_df = resample(new_target_df[ is_target ],
                       replace=False,
                       n_samples=num_targets,
                       random_state=42)

minority_df = resample(new_target_df[ ~is_target ],
                       replace=False,
                       n_samples=num_targets,
                       random_state=42)

train_df = pd.concat([majority_df, minority_df])
train_df['target'].value_counts()

1    338409
0    338409
Name: target, dtype: int64

In [9]:
y = train_df['target']
X = train_df['comment_text']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [10]:
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [11]:
sw = stopwords.words('english')

def word_processor(comment, stopword_list=sw):
    # tokenize (simultaneously removes punctuation)
    tokenizer = RegexpTokenizer(pattern="([a-zA-Z]+(?:'[a-z]+)?)")
    words = tokenizer.tokenize(comment)

    # lowercase and remove stopwords
    words = [word.lower() for word in words if word.lower() not in stopword_list]
    
    # lemmatize (accounting for part of speech)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word[0], get_wordnet_pos(word[1])) 
             for word in pos_tag(words)]

    return words

In [14]:
X_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 507613 entries, 1483102 to 1422804
Series name: comment_text
Non-Null Count   Dtype 
--------------   ----- 
507613 non-null  object
dtypes: object(1)
memory usage: 7.7+ MB


In [15]:
train_df = pd.concat([y_train, X_train], axis=1)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 507613 entries, 1483102 to 1422804
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   target        507613 non-null  int64 
 1   comment_text  507613 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.6+ MB


In [19]:
%%time
train_df['proc_text'] = list(map(word_processor, train_df['comment_text']))

CPU times: user 13min 5s, sys: 8.65 s, total: 13min 14s
Wall time: 13min 17s


In [24]:
is_target = (train_df['target'] == 1)

for i in range(3):
    print("Target comment before tokenizing: ", train_df[is_target]['comment_text'].iloc[i])
    print("Target comment after tokenizing :", train_df[is_target]['proc_text'].iloc[i])
    print()

for i in range(3):
    print("Non-target comment before tokenizing: ", train_df[~is_target]['comment_text'].iloc[i])
    print("Non-target comment after tokenizing :", train_df[~is_target]['proc_text'].iloc[i])
    print()

Target comment before tokenizing:  Dead On Arrival......won't ever pass!
Target comment after tokenizing : ['dead', 'arrival', 'ever', 'pas']

Target comment before tokenizing:  Drain the swamp? Really?  Suckers!!!!
Target comment after tokenizing : ['drain', 'swamp', 'really', 'sucker']

Target comment before tokenizing:  I will confirm your statement.  I have worked with most of the employees that were put through hell by this government.  The thing that continues to bother me is that the lead investigator ended up being promoted and moved out of the ministry where she continues in an executive position with high pay but has no staff.  Everybody concerned that brought on this hell has ended up being protected by the liberals with promotions or nice severance packages.  Just remember when you vote in May on the type of government you want.
Target comment after tokenizing : ['confirm', 'statement', 'work', 'employee', 'put', 'hell', 'government', 'thing', 'continue', 'bother', 'lead', 

In [46]:
bully_train_df = train_df[is_target]

In [47]:
bully_train_df.head()

,target,comment_text,proc_text
1483102,1,Dead On Arrival......won't ever pass!,"[dead, arrival, ever, pas]"
132094,1,Drain the swamp? Really? Suckers!!!!,"[drain, swamp, really, sucker]"
1486986,1,I will confirm your statement. I have worked with most of the employees that were put through hell by this government. The thing that continues to bother me is that the lead investigator ended up being promoted and moved out of the ministry where she continues in an executive position with hig...,"[confirm, statement, work, employee, put, hell, government, thing, continue, bother, lead, investigator, end, promote, moved, ministry, continue, executive, position, high, pay, staff, everybody, concern, brought, hell, end, protect, liberal, promotion, nice, severance, package, remember, vote, ..."
1351710,1,"I suspect Bush's reticence to open his mouth has something to do with that old bit of folk wisdom about ""people in glass houses who've committed a slew of war crimes"" not throwing stones and all. Besides, I think it's hard for Bush to string sentences together without Cheney's hand up his backs...","[suspect, bush's, reticence, open, mouth, something, old, bit, folk, wisdom, people, glass, house, who've, commit, slew, war, crime, throw, stone, besides, think, hard, bush, string, sentence, together, without, cheney's, hand, backside]"
401223,1,"Also if there is a catastrophe which we hope will not happen, but if it does federal aid will be very slow because Ige and Chin will make this a sanctuary state and it will be there fault for the slow aid we get, or the federal aid we get some of it could be cut off because of our sanctuary stat...","[also, catastrophe, hope, happen, federal, aid, slow, ige, chin, make, sanctuary, state, fault, slow, aid, get, federal, aid, get, could, cut, sanctuary, state, status, happen, blame, clown, two, state, mayor]"


In [ ]:
%%time
bully_corpus_list = sum(bully_train_df['proc_text'], [])

In [ ]:
%%time
bully_freq_dist = FreqDist(bully_corpus_list)

In [ ]:
bully_freq = pd.Series(dict(bully_freq_dist.most_common(20)))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
bully_plot = sns.barplot(x=bully_freq.values, y=bully_freq.index, ax=ax)
plt.savefig('../images/bully_FreqDist.jpg', transparent = False, bbox_inches="tight")

In [ ]:
nonbully_train_df = train_df[~is_target]
nonbully_train_df.head()

In [ ]:
%%time
nonbully_corpus_list = sum(nonbully_train_df['proc_text'], [])

In [ ]:
%%time
nonbully_freq_dist = FreqDist(nonbully_corpus_list)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
nonbully_plot = sns.barplot(x=nonbully_freq.values, y=nonbully_freq.index, ax=ax)
plt.savefig('../images/nonbully_FreqDist.jpg', transparent = False, bbox_inches="tight")

In [ ]:
wordcloud = WordCloud(min_font_size=10, max_font_size=100, 
                      width=1000, height=1000).generate_from_frequencies(dict(bully_freq_dist))
wordcloud.to_file("../images/bully_wordcloud.png")

In [ ]:
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off");

In [ ]:
wordcloud = WordCloud(min_font_size=10, max_font_size=100, 
                      width=1000, height=1000,
                      background_color='white').generate_from_frequencies(dict(nonbully_freq_dist))
wordcloud.to_file("../images/nonbully_wordcloud.png")

In [ ]:
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off");